In [5]:
# Install PyMuPDF and pdfrw
# !pip install pymupdf pdfrw PyPDF2

In [1]:
import fitz  # PyMuPDF
from pdfrw import PdfReader, PdfWriter, IndirectPdfDict, PdfName, PdfObject, PdfDict

# Function to extract form field information from a PDF
def extract_form_fields(pdf_path):
    pdf = fitz.open(pdf_path)
    form_fields = []

    for page in pdf:
        widgets = page.widgets()  # Get the form widgets (form fields)
        if widgets:  # If form fields exist on this page
            for widget in widgets:
                field_type = widget.field_type  # Field type (checkbox, text, etc.)
                field_name = widget.field_name  # Field name
                try:
                    field_value = widget.get_text("text")  # Field value
                except:
                    field_value = None
                form_fields.append({
                    "name": field_name,
                    "type": field_type,
                    "value": field_value,
                })

    pdf.close()
    return form_fields

# Extract the form fields from your PDF
pdf_path = "/Users/tercule/Documents/GitHub/PDF_filler/data/PDF_samples/EVR-E3-Datenblatt-Speicher-V-2.pdf"
fields = extract_form_fields(pdf_path)

# Print the form fields to understand their structure
for field in fields:
    print(f"Field Name: {field['name']}, Field Type: {field['type']}, Field Value: {field['value']}")

        


Field Name: Name, Field Type: 7, Field Value: None
Field Name: Strasse, Field Type: 7, Field Value: None
Field Name: PLZ ORT, Field Type: 7, Field Value: None
Field Name: , Field Type: 5, Field Value: None
Field Name: Datenblatt_Speichersystem[0].Datenblatt_Seite1[0].Anlagenbetreiber_Anschlussprojekt[0].Anlagen_Anfrage_Nr[0].Optionsfeld[0], Field Type: 5, Field Value: None
Field Name: Angabe, Field Type: 7, Field Value: None
Field Name: Anzahl, Field Type: 7, Field Value: None
Field Name: Check Box1, Field Type: 2, Field Value: None
Field Name: Check Box2, Field Type: 2, Field Value: None
Field Name: Check Box3, Field Type: 2, Field Value: None
Field Name: Check Box4, Field Type: 2, Field Value: None
Field Name: Check Box5, Field Type: 2, Field Value: None
Field Name: Check Box6, Field Type: 2, Field Value: None
Field Name: Check Box7, Field Type: 2, Field Value: None
Field Name: Check Box8, Field Type: 2, Field Value: None
Field Name: Nutzbare, Field Type: 7, Field Value: None
Field N

In [44]:
# Create a dictionary with field names and their values to fill
import fitz  # PyMuPDF
from pdfrw import PdfReader, PdfWriter, IndirectPdfDict, PdfName, PdfObject, PdfDict
from PyPDF2.generic import NameObject
field_values = {
    "(Check 55)": "/Yes",  # Example checkbox (use "Yes" or "No" to check/uncheck)
    "(Check 55)": "/Yes",  # Example text field
}

# Function to fill form fields in a PDF
def fill_form_fields(input_pdf_path, output_pdf_path, field_values):
    pdf = PdfReader(input_pdf_path)

    for page in pdf.pages:
        annotations = page['/Annots']
        # print(annotations)
        for annotation in annotations:
            if annotation['/Subtype'] == '/Widget':  # It's a form field
                if annotation.get(PdfName.T) in ['Check 55']:
                    print(annotation.keys())
                    print("\tT:",annotation.get(PdfName.T))
                    print("\tV:",annotation.get(PdfName.V))
                    print("\tAS:",annotation.get(PdfName.AS))
                    print("\tAP:",annotation.get(PdfName.AP))
                    print("\tFT:",annotation.get(PdfName.FT))                
                    print("\tFf:",annotation.get(PdfName.Ff)) 
                    print("\tP:",annotation.get(PdfName.P)[PdfName.AP])                
               
                    print("\n\n")

                    annotation[PdfName.V] = "/Yes"# PdfObject("Yes")
                    annotation[PdfName.AS] = "/Yes" #PdfObject("Yes")
                    annotation.get(PdfName.P)[PdfName.AS] = "/Yes" #PdfObject("Yes")



                # field_name = annotation.get('/T')  # Field name
                
                
                # if annotation.get(PdfName.FT) == PdfName.Tx:
                #     annotation[PdfName.V] = field_name
                # if annotation.get(PdfName.FT) == PdfName.Tx:
                #     if "Field Name" == "Text1": 
                #         for text in df["Text1"]:
                #             print("Text1"[text])
        
                    
                #     # Properly set the field value
                #     annotation[PdfName.V] = "Raphita comeme los huevos" #PdfObject()

                # if annotation.get(PdfName.FT) == PdfName.Btn:
                        # print("Annotation:", annotation)
                        # print(annotation.get(PdfName.T))
                        # print("\tV:",annotation.get(PdfName.V))
                        # print("\tAS:",annotation.get(PdfName.AS))
                        # print("\tAP:",annotation.get(PdfName.AP))
                        # print("\tFf:",annotation.get(PdfName.Ff))


                        # # Properly set the checkbox value
                        # annotation[PdfName.Ff] = 0
                        # annotation[PdfName.V] = NameObject(PdfName.Yes) 
                        # annotation[PdfName.AS] = NameObject(PdfName.Yes)
                        # print("New Annotation:", annotation)
                        # print("\tV:",annotation.get(PdfName.V))
                        # print("\tAS:",annotation.get(PdfName.AS))
                        # print("\tAP:",annotation.get(PdfName.AP))
                        # print("\n\n")


    # Write to the output PDF
    PdfWriter().write(output_pdf_path, trailer=pdf)
    

from pdfrw import PdfReader, PdfWriter, PdfName, PdfObject

# Function to fill form fields in a PDF
def fill_form_fields(input_pdf_path, output_pdf_path, field_values):
    pdf = PdfReader(input_pdf_path)

    for page in pdf.pages:
        annotations = page['/Annots']
        if annotations is not None:  # Check if annotations exist
            for annotation in annotations:
                if annotation['/Subtype'] == '/Widget':  # It's a form field
                    field_name = annotation.get(PdfName.T)
                    if field_name in field_values:
                        field_value = field_values[field_name]
                        field_type = annotation.get(PdfName.FT)
                        
                        if field_type == PdfName.Btn:  # Button field, likely a checkbox
                            annotation[PdfName.V] = field_value
                            annotation[PdfName.AS] = field_value
                        elif field_type == PdfName.Tx:  # Text field
                            annotation.update(
                                PdfDict(V=PdfObject(field_value))
                            )
                        # Add more field type checks as needed

    # Write to the output PDF
    PdfWriter().write(output_pdf_path, trailer=pdf)

# Fill the form fields with specified values
input_pdf = "/Users/tercule/Documents/GitHub/PDF_filler/data/sample.pdf"
output_pdf = "data/template.pdf"
field_values_3 = {
    "(Check 55)": "/Yes",
    "(Check 55)": "/Yes",
}

fill_form_fields(input_pdf, output_pdf, field_values)

print("Filled form has been created:", output_pdf)




# Fill the form fields with specified values
input_pdf = "/Users/tercule/Documents/GitHub/PDF_filler/data/sample.pdf"
output_pdf = "data/template.pdf"

fill_form_fields(input_pdf, output_pdf, field_values)

print("Filled form has been created:", output_pdf)


Filled form has been created: data/template.pdf
Filled form has been created: data/template.pdf


In [43]:
field_values_3

{'(Check 55)': '/Yes'}

In [33]:
from pdfrw import PdfReader, PdfWriter, PdfName, PdfObject

# Function to fill form fields in a PDF
def fill_form_fields(input_pdf_path, output_pdf_path, field_values):
    pdf = PdfReader(input_pdf_path)

    for page in pdf.pages:
        annotations = page['/Annots']
        if annotations is not None:  # Check if annotations exist
            for annotation in annotations:
                if annotation['/Subtype'] == '/Widget':  # It's a form field
                    field_name = annotation.get(PdfName.T)
                    if field_name in field_values:
                        field_value = field_values[field_name]
                        field_type = annotation.get(PdfName.FT)
                        
                        if field_type == PdfName.Btn:  # Button field, likely a checkbox
                            annotation[PdfName.V] = field_value
                            annotation[PdfName.AS] = field_value
                        elif field_type == PdfName.Tx:  # Text field
                            annotation.update(
                                PdfDict(V=PdfObject(field_value))
                            )
                        # Add more field type checks as needed

    # Write to the output PDF
    PdfWriter().write(output_pdf_path, trailer=pdf)

# Fill the form fields with specified values
input_pdf = "/Users/tercule/Documents/GitHub/PDF_filler/data/sample.pdf"
output_pdf = "data/template.pdf"
field_values = {
    "(Check 55)": "/Yes",
    "(Check 55)": "/Yes",
}

fill_form_fields(input_pdf, output_pdf, field_values)

print("Filled form has been created:", output_pdf)


Filled form has been created: data/template.pdf


In [36]:
field_values

{'(Check 55)': '/Yes'}

In [34]:
from pdfrw import PdfReader, PdfWriter, PdfName, PdfObject, PdfDict

# Function to fill form fields in a PDF
def fill_form_fields(input_pdf_path, output_pdf_path, field_values):
    pdf = PdfReader(input_pdf_path)

    for page in pdf.pages:
        annotations = page['/Annots']
        if annotations is not None:  # Check if annotations exist
            for annotation in annotations:
                if annotation['/Subtype'] == '/Widget':  # It's a form field
                    field_name = annotation.get(PdfName.T)
                    if field_name in field_values:
                        field_value = field_values[field_name]
                        field_type = annotation.get(PdfName.FT)
                        
                        # Print detailed information about the checkbox annotation
                        if field_name == '(Check 55)':
                            print(f"Annotation for {field_name}:")
                            for key in annotation.keys():
                                print(f"{key}: {annotation[key]}")
                            print("\n")

                        if field_type == PdfName.Btn:  # Button field, likely a checkbox
                            annotation[PdfName.V] = field_value
                            annotation[PdfName.AS] = field_value
                        elif field_type == PdfName.Tx:  # Text field
                            annotation.update(
                                PdfDict(V=PdfObject(field_value))
                            )
                        # Add more field type checks as needed

    # Write to the output PDF
    PdfWriter().write(output_pdf_path, trailer=pdf)

# Fill the form fields with specified values
input_pdf = "/Users/tercule/Documents/GitHub/PDF_filler/data/sample.pdf"
output_pdf = "data/template.pdf"
field_values = {
    "(Check 55)": "/Yes",
}

fill_form_fields(input_pdf, output_pdf, field_values)

print("Filled form has been created:", output_pdf)


Annotation for (Check 55):
/Border: ['0', '0', '0.5']
/Rect: ['518.874', '515.5763', '533.0472', '529.7495']
/T: (Check 55)
/F: 4
/BS: {'/W': '0.5'}
/Subtype: /Widget
/DA: (/ZapfDingbatsITC 12 Tf 0 g)
/MK: {'/BC': ['0.7530000', '0.7530000', '0.7530000'], '/BG': ['1', '1', '1'], '/CA': '(8)'}
/AP: {'/N': (427, 0), '/D': (428, 0)}
/H: /P
/V: /Off
/M: (D:20190930140358Z00'00')
/AS: /Off
/FT: /Btn
/Type: /Annot
/Ff: 0
/Page: 0


Filled form has been created: data/template.pdf


In [35]:
from pdfrw import PdfReader, PdfWriter, PdfName, PdfObject

# Function to fill checkboxes in a PDF
def fill_checkboxes(input_pdf_path, output_pdf_path, field_values):
    pdf = PdfReader(input_pdf_path)

    for page in pdf.pages:
        if '/Annots' not in page:
            continue  # No annotations on this page

        annotations = page['/Annots']
        for annotation in annotations:
            if annotation.get(PdfName.Subtype) == PdfName.Widget:
                field_name = annotation.get(PdfName.T)  # Field name
                if field_name and field_name in field_values:
                    field_value = field_values[field_name]

                    # If it's a checkbox (button type), set it accordingly
                    if annotation.get(PdfName.FT) == PdfName.Btn:
                        # Check if it's a checkbox with multiple appearances
                        # Find the default value key in the dictionary
                        normal_appearance = annotation.get(PdfName.AP, {}).get(PdfName.N, {})
                        
                        # Check if the desired value matches an existing key in the appearance
                        if field_value == 'checked':
                            if PdfName.On in normal_appearance:
                                annotation[PdfName.V] = 
                                annotation[PdfName.AS] = PdfName.On
                        else:  # 'unchecked'
                            annotation[PdfName.V] = PdfName.Off
                            annotation[PdfName.AS] = PdfName.Off

    # Write the updated PDF
    PdfWriter(output_pdf_path, trailer=pdf).write()

# Define the checkbox values to set
field_values = {
    '(Check Box5)': 'checked',  # This checkbox should be checked
    '(Check Box6)': 'unchecked',  # This checkbox should be unchecked
}

input_pdf = "data/sample.pdf"
output_pdf = "data/sample_filled2.pdf"

# Fill the checkboxes in the PDF
fill_checkboxes(input_pdf, output_pdf, field_values)

print("Filled PDF created:", output_pdf)


SyntaxError: invalid syntax (734467338.py, line 27)

In [48]:
from pdfrw import PdfReader, PdfWriter, PdfName, PdfObject, PdfDict

# Function to fill form fields in a PDF
def fill_form_fields(input_pdf_path, output_pdf_path, field_values):
    pdf = PdfReader(input_pdf_path)

    for page in pdf.pages:
        annotations = page['/Annots']
        if annotations is not None:  # Check if annotations exist
            for annotation in annotations:
                if annotation['/Subtype'] == '/Widget':  # It's a form field
                    field_name = annotation.get(PdfName.T)
                    if field_name in field_values:
                        field_value = field_values[field_name]
                        field_type = annotation.get(PdfName.FT)
                        
                        # Print detailed information about the checkbox annotation
                        if field_name == '(Check Box26)':
                            print(f"Annotation for {field_name}:")
                            for key in annotation.keys():
                                print(f"{key}: {annotation[key]}")
                            print("\n")

                        if field_type == PdfName.Btn:  # Button field, likely a checkbox
                            annotation[PdfName.V] = field_value
                            annotation[PdfName.AS] = field_value
                        elif field_type == PdfName.Tx:  # Text field
                            annotation.update(
                                PdfDict(V=PdfObject(field_value))
                            )
                        # Add more field type checks as needed

    # Write to the output PDF
    PdfWriter().write(output_pdf_path, trailer=pdf)

# Fill the form fields with specified values
input_pdf = "/Users/tercule/Documents/GitHub/PDF_filler/data/PDF_samples/E-2-Datenblatt-fuer-Erzeugungseinheiten.pdf"
output_pdf = "data/template.pdf"
field_values = {
    "(Check Box26)": "/Yes",
}

fill_form_fields(input_pdf, output_pdf, field_values)

print("Filled form has been created:", output_pdf)


Annotation for (Check Box26):
/Border: ['0', '0', '1']
/Rect: ['122.277', '685.387', '133.839', '697.65']
/T: (Check Box26)
/F: 4
/Subtype: /Widget
/DA: (/Helvetica 12 Tf 0 g)
/MK: {'/BC': ['0'], '/BG': ['1']}
/AP: {'/N': (389, 0), '/D': (390, 0)}
/H: /P
/V: /Ja
/AS: /Ja
/FT: /Btn
/Type: /Annot
/Ff: 0


Filled form has been created: data/template.pdf


In [36]:
import pandas as pd 
df = pd.read_excel("data/Rafa_pdf.xlsx", header = 3)


df

,Text1,Text2,Text3,Text4,Text5,Text6,Text7,Text8,Check7,Unnamed: 9,...,Text19,Unnamed: 31,Unnamed: 32,Unnamed: 33,Unnamed: 34,Unnamed: 35,Text20,Text21,Text22,Text999
0,Tereza A,Feria,Orat,Orat,603,73,80,80,Y,N,...,80,N,N,N,Y,N,Text20-a,Text21-a,"Orta, 22.4",89
1,Alberto,Violna,Orat,Orta,605,64,69,80,Y,N,...,70,N,N,Y,N,N,Text20-b,Text21-b,"Orta, 22.4",79
2,Fdsjfk,Fd,Orta,Orta,603,65,60,70,Y,Y,...,40,N,N,Y,N,N,Text20-c,Text21-c,"Orta, 22.4",239
3,dfjldfs,Dfgd,Orta,Orat,606,65,60,50,N,Y,...,20,Y,N,Y,N,N,Text20-d,Text21-d,"Orta, 22.4",134
4,Fdjfks,Fdfd,Df,Orta,606,65,80,50,Y,Y,...,20,Y,N,Y,N,N,Text20-e,Text21-e,"Orta, 22.4",890


In [18]:
from pdfrw import PdfReader, PdfWriter, PdfName, PdfObject, PdfDict

# Function to fill form fields in a PDF
def fill_form_fields(input_pdf_path, output_pdf_path, field_values):
    pdf = PdfReader(input_pdf_path)

    for page in pdf.pages:
        annotations = page['/Annots']
        if annotations is not None:  # Check if annotations exist
            for annotation in annotations:
                if annotation['/Subtype'] == '/Widget':  # It's a form field
                    field_name = annotation.get(PdfName.T)
                    if field_name in field_values:
                        field_value = field_values[field_name]
                        field_type = annotation.get(PdfName.FT)

                        # Print detailed information about the checkbox annotation
                        if field_name == '(Check Box10)':
                            print(f"Annotation for {field_name}:")
                            for key in annotation.keys():
                                print(f"{key}: {annotation[key]}")
                            print("\n")

                        if field_type == PdfName.Btn:  # Button field, likely a checkbox
                            if field_value.lower() == "yes":
                                annotation[PdfName.V] = PdfName('/Yes')
                                annotation[PdfName.AS] = PdfName('/Yes')
                            elif field_value.lower() == "no":
                                annotation[PdfName.V] = PdfName('/Off')
                                annotation[PdfName.AS] = PdfName('/Off')
                        elif field_type == PdfName.Tx:  # Text field
                            annotation.update(
                                PdfDict(V=PdfObject(field_value))
                            )
                        # Add more field type checks as needed

    # Write to the output PDF
    PdfWriter().write(output_pdf_path, trailer=pdf)

# Fill the form fields with specified values
input_pdf = "/Users/tercule/Documents/GitHub/PDF_filler/data/PDF_samples/EVR-E3-Datenblatt-Speicher-V-2.pdf"
output_pdf = "data/template.pdf"
field_values = {
    "(Check Box10)": "Yes",  # Use Yes to check and No to uncheck the checkbox
}

fill_form_fields(input_pdf, output_pdf, field_values)

print("Filled form has been created:", output_pdf)


Annotation for (Check Box10):
/Border: ['0', '0', '0']
/Rect: ['497.835', '571.284', '507.835', '581.284']
/T: (Check Box10)
/F: 4
/BS: {'/W': '0'}
/Subtype: /Widget
/DA: (/Helvetica 8 Tf 0 g)
/MK: {'/CA': '(8)'}
/TU: (ja)
/AP: {'/N': (481, 0), '/D': (482, 0)}
/H: /P
/V: /1
/StructParent: 18
/AS: /1
/FT: /Btn
/Type: /Annot
/Ff: 0


Filled form has been created: data/template.pdf


['Text1',
 'Text2',
 'Text3',
 'Text4',
 'Text5',
 'Text6',
 'Text7',
 'Text10',
 'Text12',
 'Text20',
 'Check Box5',
 'Check Box6',
 'Check Box7',
 'Check Box8',
 'Button1',
 'Button2',
 'Check7',
 'Check 8',
 'Text8',
 'Check 800',
 'Text9',
 'Check 54',
 'Check 55',
 'Check 540',
 'Check 551',
 'Check 002',
 'Text22',
 'Text21',
 'Text11',
 'Check 002222',
 'Check 00222',
 'Check 012022',
 'Check 00223',
 'Check 00722',
 'Check 07022',
 'Text14',
 'Text17',
 'Text18',
 'Text15',
 'Text16',
 'Text19',
 'Check 7022',
 'Text999']

In [112]:
form_fields = list(fillpdfs.get_form_fields("data/sample.pdf").keys()) # to get the names of the boxes that we need to fill 
[x for x in form_fields if "Check" in x]

['Check Box5',
 'Check Box6',
 'Check Box7',
 'Check Box8',
 'Check7',
 'Check 8',
 'Check 800',
 'Check 54',
 'Check 55',
 'Check 540',
 'Check 551',
 'Check 002',
 'Check 002222',
 'Check 00222',
 'Check 012022',
 'Check 00223',
 'Check 00722',
 'Check 07022',
 'Check 7022']

In [114]:
from fillpdf import fillpdfs
import pandas as pd 
import os 


# form_fields = list(fillpdfs.get_form_fields("data/sample.pdf").keys()) # to get the names of the boxes that we need to fill 
df = pd.read_excel("data/Rafa_pdf.xlsx", header = 3) # we import the data 

input_pdf_path = "data/sample.pdf"
output_directory = "Carpeta_PDF"
if output_directory not in os.listdir():
    os.makedirs(output_directory)

for index, row in df.iterrows():

    data_dic = row.to_dict()
    data_dic = {k: "Yes" if v == "Y" else v for k, v in data_dic.items()}
    data_dic = {k: "Off" if v == "N" else v for k, v in data_dic.items()}

    print(data_dic)
    filename_ext = "_".join(data_dic['Text1'].split())
    output_pdf_path = os.path.join(output_directory, f"output_{filename_ext}.pdf" )#f"{data_dic['Text1']}.pdf")  # Construct the output PDF path dynamically

    # Call the write_fillable_pdf function with the correct arguments
    fillpdfs.write_fillable_pdf(input_pdf_path=input_pdf_path, output_pdf_path=output_pdf_path, data_dict=data_dic)


{'Text1': 'Tereza A', 'Text2': 'Feria', 'Text3': 'Orat', 'Text4': 'Orat', 'Text5': 603, 'Text6': 73, 'Text7': 80, 'Text8': 80, 'Check7': 'Off', 'Check 8': 'Yes', 'Check 800': 'Off', 'Text9': 90, 'Check 55': 'Yes', 'Check 54': 'Yes', 'Check 551': 'Off', 'Check 540': 'Off', 'Text10': 90, 'Text11': 60, 'Check 002222': 'Off', 'Check 00222': 'Off', 'Check 002': 'Off', 'Text12': 90, 'Check 012022': 'Off', 'Check 00223': 'Yes', 'Check 00722': 'Off', 'Check 07022': 'Off', 'Text14': 90, 'Text15': 80, 'Text16': 80, 'Text17': 80, 'Text18': 80, 'Text19': 80, 'Check 7022': 'Off', 'Check Box8': 'Off', 'Check Box5': 'Off', 'Check Box6': 'Yes', 'Check Box7': 'Off', 'Text20': 'Text20-a', 'Text21': 'Text21-a', 'Text22': 'Orta, 22.4', 'Text999': 89}
{'Text1': 'Alberto', 'Text2': 'Violna', 'Text3': 'Orat', 'Text4': 'Orta', 'Text5': 605, 'Text6': 64, 'Text7': 69, 'Text8': 80, 'Check7': 'Off', 'Check 8': 'Yes', 'Check 800': 'Off', 'Text9': 80, 'Check 55': 'Yes', 'Check 54': 'Yes', 'Check 551': 'Off', 'Check

In [80]:
from fillpdf import fillpdfs
import pandas as pd 
import os 


form_fields = list(fillpdfs.get_form_fields("data/sample.pdf").keys()) # to get the names of the boxes that we need to fill 
df = pd.read_excel("data/Rafa_pdf.xlsx", header = 3) # we import the data 

data_dic = {}
row = df.iloc[0]
print(row)
# for index, row in df.iterrows():
#     if index == 0:
#         # Text1 = row["Text1"]
#         # Text2 = row["Text2"]
#         # Text3 = row["Text3"]
#         # Text4 = row["Text4"]
#         # Text5 = row["Text5"]
#         # Text6 = row["Text6"]
#         # Text7 = row["Text7"]
#         # Text8 = row["Text8"]
#         # Text9 = row["Text9"]
#         # Text10 = row["Text10"]
#         # Text11 = row["Text11"]
#         # Text12 = row["Text12"]
#         # Text14 = row["Text14"]
#         # Text15 = row["Text15"]
#         # Text16 = row["Text16"]
#         # Text17 = row["Text17"]
#         # Text18 = row["Text18"]
#     else:
#         break
#     #Check7 = '/Yes' #if row["Check7"] == "Y" else '/Off' 

data_dic = {
    form_fields[0] : Text1,
    form_fields[1] : Text2, 
    form_fields[2] : Text3,
    form_fields[3] : Text4,
    form_fields[4] : Text5,
    form_fields[5] : Text6,
    form_fields[6] : Text7,
    form_fields[18] : Text8,
    form_fields[20] : Text9,
    form_fields[7] : Text10,
    form_fields[28] : Text11,
    form_fields[8] : Text12,
    form_fields[35] : Text14,
    form_fields[38] : Text15,
    form_fields[39] : Text16,
    form_fields[36] : Text17,
    form_fields[37] : Text18
   # form_fields[16] : Check7 #'/Yes' if row["Check7"] == "Y" else '/Off'

}
print(form_fields)

fillpdfs.write_fillable_pdf(input_pdf_path="data/sample.pdf", output_pdf_path="new.pdf",os.path.join("Carpeta_PDF", f'{"Text1"}.pdf') data_dict= data_dic)

['Text1', 'Text2', 'Text3', 'Text4', 'Text5', 'Text6', 'Text7', 'Text10', 'Text12', 'Text20', 'Check Box5', 'Check Box6', 'Check Box7', 'Check Box8', 'Button1', 'Button2', 'Check7', 'Check 8', 'Text8', 'Check 800', 'Text9', 'Check 54', 'Check 55', 'Check 540', 'Check 551', 'Check 002', 'Text22', 'Text21', 'Text11', 'Check 002222', 'Check 00222', 'Check 012022', 'Check 00223', 'Check 00722', 'Check 07022', 'Text14', 'Text17', 'Text18', 'Text15', 'Text16', 'Text19', 'Check 7022', 'Text999']


In [52]:
import pandas as pd
from PyPDF2 import PdfReader, PdfWriter
from PyPDF2.generic import NameObject, TextStringObject

# Function to fill PDF form fields with data dictionary
def fill_pdf_form(input_pdf_path, output_pdf_path, data_dict):
    with open(input_pdf_path, "rb") as pdf_file:
        pdf_reader = PdfReader(pdf_file)
        pdf_writer = PdfWriter()

        # Loop through each page of the PDF
        for page_num in range(len(pdf_reader.pages)):
            page = pdf_reader.pages[page_num]
            if '/Annots' in page:
                for annot in page['/Annots']:
                    obj = annot.get_object()
                    if '/T' in obj and obj['/T'] in data_dict:
                        # Fill form field with data from dictionary
                        field_name = obj['/T']
                        if field_name in data_dict:
                            field_value = data_dict[field_name]
                            obj.update({
                                NameObject("/V"): TextStringObject(field_value)
                            })
            pdf_writer.add_page(page)

        # Write filled PDF to output file
        with open(output_pdf_path, "wb") as output_file:
            pdf_writer.write(output_file)

# Define PDF form field names and corresponding data mappings
form_fields = list(fillpdfs.get_form_fields("data/sample.pdf").keys())
df = pd.read_excel("data/Rafa_pdf.xlsx", header=3)

for index, row in df.iterrows():
    Text1 = row["Text1"]
    Text2 = row["Text2"]
    Text3 = row["Text3"]
    Check7 = '/Yes' if row["Check7"] == "Y" else '/Off'

    # Construct data dictionary for filling PDF form fields
    data_dic = {
        form_fields[0]: Text1,
        form_fields[1]: Text2,
        form_fields[2]: Text3,
        form_fields[16]: Check7  # Assuming form_fields[16] corresponds to 'Check7' field
    }

    # Generate filled PDF with current data dictionary
    output_pdf_path = f"output_{index}.pdf"
    fill_pdf_form(input_pdf_path="data/sample.pdf", output_pdf_path=output_pdf_path, data_dict=data_dic)
